In [52]:
import pandas as pd 
import numpy as np
import yfinance as yf

In [53]:
df = yf.download('BUMI.jk',
                 start='2010-12-31', 
                 end='2019-12-31',
                 actions='inline',
                 progress=False)

df = df.loc[:, ['Close']]
df.rename(columns={'Close':'close'}, inplace=True)

In [54]:
df['simple_rtn'] = df.close.pct_change()
df['log_rtn'] = np.log(df.close/df.close.shift(1))

In [55]:
df.head()

,close,simple_rtn,log_rtn
Date,,,
2011-01-03,3125.0,NaN,NaN
2011-01-04,3250.0,0.040000,0.039221
2011-01-05,3275.0,0.007692,0.007663
2011-01-06,3275.0,0.000000,0.000000
2011-01-07,3125.0,-0.045802,-0.046884


In [56]:
import pandas as pd 
import quandl
import seaborn as sns


QUANDL_KEY = 'et7jqG6esE8zbud7VEaM' # replace {key} with your own API key  
quandl.ApiConfig.api_key = QUANDL_KEY

In [57]:
df_all_dates = pd.DataFrame(index=pd.date_range(start='2010-12-31', 
                                                end='2019-12-31'))

df = df_all_dates.join(df[['close']], how='left') \
                 .fillna(method='ffill') \
                 .asfreq('A')

In [58]:
df_all_dates.head()

""
2010-12-31
2011-01-01
2011-01-02
2011-01-03
2011-01-04


In [59]:
df.head()

,close
2010-12-31,NaN
2011-12-31,2175.0
2012-12-31,590.0
2013-12-31,300.0
2014-12-31,80.0


In [60]:
df_cpi = quandl.get("ODA/IDN_PCPI", 
                    authtoken="et7jqG6esE8zbud7VEaM", 
                    start_date='2010-12-31', 
                    end_date='2019-12-31')
df_cpi.rename(columns={'Value':'cpi'}, inplace=True)

In [61]:
df_cpi.head()

,cpi
Date,
2010-12-31,91.293
2011-12-31,96.171
2012-12-31,99.999
2013-12-31,106.413
2014-12-31,113.218


In [62]:
df_merged = df.join(df_cpi, how='right')

In [63]:
df_merged.head()

,close,cpi
Date,,
2010-12-31,NaN,91.293
2011-12-31,2175.0,96.171
2012-12-31,590.0,99.999
2013-12-31,300.0,106.413
2014-12-31,80.0,113.218


In [64]:
df_merged['simple_rtn'] = df_merged.close.pct_change()
df_merged['inflation_rate'] = df_merged.cpi.pct_change()

In [65]:
df_merged.head(10)

,close,cpi,simple_rtn,inflation_rate
Date,,,,
2010-12-31,NaN,91.293,NaN,NaN
2011-12-31,2175.0,96.171,NaN,0.053432
2012-12-31,590.0,99.999,-0.728736,0.039804
2013-12-31,300.0,106.413,-0.491525,0.064141
2014-12-31,80.0,113.218,-0.733333,0.063949
2015-12-31,50.0,120.422,-0.375000,0.063629
2016-12-31,278.0,124.668,4.560000,0.035259
2017-12-31,270.0,129.416,-0.028777,0.038085
2018-12-31,103.0,133.555,-0.618519,0.031982


In [66]:
df_merged['real_rtn'] = (df_merged.simple_rtn + 1 ) / (df_merged.inflation_rate + 1) - 1
df_merged.head(10)

,close,cpi,simple_rtn,inflation_rate,real_rtn
Date,,,,,
2010-12-31,NaN,91.293,NaN,NaN,NaN
2011-12-31,2175.0,96.171,NaN,0.053432,NaN
2012-12-31,590.0,99.999,-0.728736,0.039804,-0.739120
2013-12-31,300.0,106.413,-0.491525,0.064141,-0.522174
2014-12-31,80.0,113.218,-0.733333,0.063949,-0.749361
2015-12-31,50.0,120.422,-0.375000,0.063629,-0.412389
2016-12-31,278.0,124.668,4.560000,0.035259,4.370635
2017-12-31,270.0,129.416,-0.028777,0.038085,-0.064409
2018-12-31,103.0,133.555,-0.618519,0.031982,-0.630341


In [67]:
df = df_merged['close'].dropna()
df.head(10)

Date
2011-12-31    2175.0
2012-12-31     590.0
2013-12-31     300.0
2014-12-31      80.0
2015-12-31      50.0
2016-12-31     278.0
2017-12-31     270.0
2018-12-31     103.0
2019-12-31      66.0
Name: close, dtype: float64

In [68]:
x = df[0]
print(x)
y = df[-1]
print(y)
year = len(df)
print(year)
inflation = df_merged['inflation_rate'].sum()
print(inflation)

Return = (y-x)/x 

if Return > 0:
    Return - inflation

else:
    Return = Return 

print(Return)

2175.0
66.0
9
0.4237290291701006
-0.9696551724137931
